In [1]:
import numpy as np
import pandas as pd
import pickle
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
#from joblib import dump, load

markdown + comments 1
code 0.4
simpele 0.3
score = [s1,s2,s3]
dot weights
query en ding cosine similarity
vector cosine similarity new gewogen score
Lineaire combinatie

In [2]:
# connect to local elastic search host
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST], timeout=30) # , max_retries=10, retry_on_timeout=True
INDEX2 = 'info_clone2'

In [3]:
es.ping()

True

In [5]:
s = Search(using=es, index=INDEX2)
q = Q("wildcard", name='*')
query = s.query(q).extra(size=10000)
response = query.execute()
print(len(response))

10000


In [6]:
# get the ids
ids = []
for res in response:
    nid = res.meta.id
    ids.append(nid)

# get the code and markdown samples
Xy = []
for i in ids:
    c = es.get(index="code", id=i)
    code = c['_source']['code']
    m = es.get(index="markdown", id=i)
    markdown = m['_source']['markdown']
    if code != [] and markdown != []:
        Xy.append((i, ''.join(code), 'code'))
        Xy.append((i, ''.join(markdown), 'markdown'))

In [7]:
Xydf = pd.DataFrame(Xy, columns=('id','sample','label'))

In [8]:
Xydf

,id,sample,label
0,479522,# requirement : 1 = must visit; 0 = must not v...,code
1,479522,### CS/ECE/ISyE 524 &mdash; Introduction to Op...,markdown
2,581536,!sudo pip install geocoderimport geocoderloc =...,code
3,581536,# Geocoding\n\n* Converting a plain text addre...,markdown
4,400611,"# Import Numpy, TensorFlow, TFLearn, and MNIST...",code
...,...,...,...
13715,475366,"__N.B.,__ Cannot use 32-bit programmable inter...",markdown
13716,435782,import sympy as sp\nimport numpy as np\nimport...,code
13717,435782,"# Integration using libraries, and flow plots\...",markdown
13718,518507,"from math import pi,sqrt\nfrom __future__ impo...",code


In [9]:
isinstance(Xydf['sample'][0], str)

True

In [56]:
# calculate TF-IDF values
def get_features(df,field,train_data,test_data):
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95)
    tfidf_vectorizer.fit_transform(train_data[field].values)

    train_feature_set = tfidf_vectorizer.transform(train_data[field].values)
    test_feature_set = tfidf_vectorizer.transform(test_data[field].values)

    return train_feature_set,test_feature_set,tfidf_vectorizer

In [11]:
# generate a training and test set, seed for same result
train_data, test_data = train_test_split(Xydf,random_state = 2000)
 
# get the training and test set labels 
Y_train = train_data['label'].values
Y_test = test_data['label'].values
     
# get the features
X_train,X_test,feat_trans = get_features(Xydf,'sample',train_data,test_data)

In [60]:
# vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
clf = LogisticRegression(random_state=0)
clf.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [62]:
# save the model to disk
filename = 'model.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [12]:
# load the model from disk
filename = 'model.pkl'
clf = pickle.load(open(filename, 'rb'))
score = clf.score(X_test, Y_test)
print(score)

0.9603498542274053


In [18]:
# get a random document
t = es.search(index="code", body={"size": 1, "query": {"function_score": {"random_score": {"seed": 10}}}})

In [42]:
sample = t['hits']['hits'][0]['_source']['code']
sample = ''.join(sample)
print(sample)

%pylab
import astropy.io.fits as pf
import pyqtgraph as pqrun /Users/aaronresnick/Desktop/CODERepository/pteropractice/winview_v02.pyflist = loadtxt('/Users/aaronresnick/Desktop/CODERepository/Doc_calibrations/flist.txt',dtype='str')flistlist = []for i in range(len(flist)):
     list.append(load_spe('/Users/aaronresnick/Desktop/CODERepository/Doc_calibrations/'+flist[i]))dark_1us = load_spe('/Users/aaronresnick/Desktop/CODERepository/Doc_calibrations/20150618_2.SPE')dark_1us.shapesuperdark = mean(dark_1us, axis=0)superdark.shapeimshow(superdark)plot(superdark[300,:])dark_2s = load_spe('/Users/aaronresnick/Desktop/CODERepository/Doc_calibrations/20150618_5.SPE')superdark2 = mean(dark_2s,axis = 0)imshow(superdark2)figure(2)
plot(superdark2[300,:])figure(3)
plot(superdark[300,:])oneSeccube = list[5]secMed = median(oneSeccube)secMedsecMicro = median(dark_1us)secMicrosec2s = median(dark_2s)sec2smilSec = list[4]milSecMed = median(milSec)milSecMedkey = array((1.0e-6,1.0e-3,1.0,2.0))meds = arr

In [60]:
sample2 = sample
sampletrans = feat_trans.transform([sample])

In [61]:
clf.predict(sampletrans)

array(['code'], dtype=object)